In [1]:
import csv
import xml.etree.ElementTree as ET

In [2]:
class Block:
    
    def __init__(self, name, x, y, typ, node):
        self.name = name
        self.x = x
        self.y = y
        self.type = typ
        self.node = node

In [3]:
def get_place_data(testbench_name):
    
    testbench = testbench_name + ".v/common"
    
    f2 = open(testbench + "/" + testbench_name + ".place", "r")
    
    b = f2.readlines()[5:]
    
    place_matrix = []
    for i in b:
        add = i.strip().split('\t')
        if '' in add:
            del add[1]
        place_matrix.append(add)
    
    place_dict = {idx[0] : idx[1:] for idx in place_matrix}
    
    return place_dict

In [4]:
def get_block_list(testbench_name, place_dict):
    
    testbench = testbench_name + ".v/common"

    tree = ET.parse(testbench + "/" + testbench_name + ".net")
    root = tree.getroot()
    
    block_list = []
    for child in root:
        if 'name' in child.attrib.keys():
            name = child.attrib['name']
            if 'io' in child.attrib['instance']:
                typ = "io"
            elif 'mult_36' in child.attrib['instance']:
                typ = "complex"
            elif 'memory' in child.attrib['instance']:
                typ = "complex"
            else:
                typ = "none"
            x = int(place_dict[child.attrib['name']][0])
            y = int(place_dict[child.attrib['name']][1])
            b = Block(name, x, y, typ, child)
            block_list.append(b)
            
    return block_list

In [5]:
def get_critical_path(testbench_name, block_list):
    
    testbench = testbench_name + ".v/common"
    
    f1 = open(testbench + "/report_timing.setup.rpt", "r")
    a = f1.readlines()[4:]
    
    crit_path_info = []
    for i in a:
        if(i == "#Path 2\n"):
            break
        else:
            crit_path_info.append(i)
            
    path = []
    for i in crit_path_info[9:]:
        if(i == "\n"):
            break
        else:
            path.append(i)
          

    
    path_matrix = []
    for i in range(len(path)):
        path_matrix.append(path[i].strip().split())

    path_matrix = path_matrix[:-1]
#     path_ll = []
    path_ll = [item[0] for item in path_matrix]
#     for item in path_matrix:
#         if(item[0] == testbench_name):
#             path_ll.append(item[1])
#             print("1")
#         else:
#             path_ll.append(item[0])
#             print(item[0] == testbench_name)
    
    
    
#     nodes_ll = [item.split('.')[0] for item in path_ll]
    nodes_ll = []
    for item in path_ll:
#         if(item.split('.')[0] == testbench_name):
#             nodes_ll.append(item.split('.')[0] + "." +item.split('.')[1])
#         else:
#             nodes_ll.append(item.split('.')[0])
#         print(item)
        a = item.split('.')
        a.pop()
        a = '.'.join(a)
#         print(a)
        nodes_ll.append(a)
    
    nodes_ll = nodes_ll[1:]
    
#     print(nodes_ll)
    
    nodes_set = []
    for i in nodes_ll:
        if i not in nodes_set:
            nodes_set.append(i)
            
    path_list = []
    for node in nodes_set:
        for block in block_list:
            for b in block.node.iter('block'):
                if(b.attrib['name'] == node):
                    path_list.append(block)
                    break
                    
    return path_list

In [6]:
def get_critical_paths(testbench_name, block_list):
    testbench = testbench_name + ".v/common"
    
    f1 = open(testbench + "/report_timing.setup.rpt", "r")
    a = f1.readlines()[4:]
    
    

    crit_path_info = []
    path_num = 0
    line_num = 0
    while(path_num < 10):
        path_num += 1
        m = []
        while(True):
            if(a[line_num] == "#Path " + str(path_num+1) +"\n"):
                break
            else:
                m.append(a[line_num])
                line_num += 1
        crit_path_info.append(m)

    slacks =[]
    k = 0    
    for path_no in range(10):
        
        while(True):
#             print(k)
            i = a[k]
            i = i.strip().split()
            k += 1
            if "slack" in i:
#                 print(i)
                slacks.append(i[2])
                break
            
        
        
    path_list = []
    for path in crit_path_info:
        k = []
        for j in path[10:]:
# #         print(j)
            if(j == "\n"):
                break
            else:
                k.append(j)
        path_list.append(k)
            
    
    
    path_matrix = []
    for i in range(len(path_list)):
        temp = []
        for j in range(len(path_list[i])):
#         print(path[i][j])
            temp.append(path_list[i][j].strip().split())
        path_matrix.append(temp)

    
    path_ll = []
    for p in path_matrix:
        k = p[:-1]
        k = [item[0] for item in k]
        path_ll.append(k)
    
    
# # #     path_ll = []
# # path_ll = [item[0] for item in path_matrix]

    path_nodes = []
    for p in path_ll:
        k = []
        for n in p:
            temp = n.split('.')
            temp.pop()
            temp = '.'.join(temp)
            k.append(temp)
        
        path_nodes.append(k)
    
    
    
    path_nodes_set = []
    for path in path_nodes:
        nodes_set = []
        for node in path:
#         print(node)
            if node not in nodes_set:
#             print(node)
                nodes_set.append(node)
#     print(nodes_set)
        path_nodes_set.append(nodes_set)

    
# pd = get_place_data("or1200")
    
    
# block_list = get_block_list("or1200", pd)
    
    path_list = []
    for nodes_set in path_nodes_set:
#     print("hello")
        temp = []
        for nodes in nodes_set:
#         print(nodes_set)
#         for node in nodes_set:
#             print(node)
                for block in block_list:
                    for b in block.node.iter('block'):
                        if(b.attrib['name'] == nodes):
                            temp.append(block)
                            break
        path_list.append(temp)
        
        
    return path_list, slacks

In [7]:
def manhattan_dist(path):
    
    distance = 0
    comp_block = 0
    locations = []
    
#     print("hello")
    for node_no in range(len(path) - 1):
#         print(path[node_no])
        distance += abs(path[node_no].x - path[node_no + 1].x) + abs(path[node_no].y - path[node_no + 1].y)
        
    for node in path:
#         print(node.name)
        if(node.type == "complex"):
            comp_block += 1
            locations.append((node.x, node.y))
            
    return distance, comp_block, locations

In [8]:
def process_tb(testbench_name):
    
    pd = get_place_data(testbench_name)
    
    bl = get_block_list(testbench_name, pd)
    
    cpl_list, slacks = get_critical_paths(testbench_name, bl)
    
    info = []
    for cpl_no in range(len(cpl_list)):
        m, c, l = manhattan_dist(cpl_list[cpl_no])
        s = slacks[cpl_no]
        info.append([m, c, l, s])
    
    return info, bl, cpl_list

In [11]:
tb_list = [
#     "arm_core",
#     "bgm",
#     "blob_merge",
#     "boundtop",
#     "ch_intrinsics",
    "diffeq1",
    "diffeq2",
#     "LU8PEEng",
#     "LU32PEEng",
#     "mcml",
#     "mkDelayWorker32B",
#     "mkPktMerge",
#     "mkSMAdapter4B",
#     "or1200",
#     "raygentop",
#     "sha",
#     "spree",
#     "stereovision0",
#     "stereovision1",
#     "stereovision2",
#     "stereovision3",
]

In [29]:
for tb in tb_list:
    info, b, c = process_tb(tb)
    print(tb + " --> done")
#     list1 = [tb, m, c, l]
#     with open(filename, "a") as fp:
    filename = tb + '.csv'
    fields = ["manhattan", "complex_blocks", "locations"]
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 
        
        # writing the fields 
        csvwriter.writerow(fields) 
        for i in range(len(info)):
            csvwriter.writerow(info[i])

diffeq1 --> done
diffeq2 --> done
